In [1]:
!pip install convokit
!pip install textstat
!pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.1/240.1 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.8/348.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from convokit import Corpus, download
import re
import numpy as np
import convokit
import spacy
from collections import Counter, defaultdict
from textstat import textstat

In [ ]:
corpus = Corpus(download("subreddit-AmItheAsshole"))

No configuration file found at /root/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
model_directory: ~/.convokit/saved-models
default_backend: mem


In [ ]:
N_SAMPLES = 10000

In [3]:
import re
import numpy as np
import convokit
import spacy
from collections import Counter, defaultdict
from textstat import textstat

# Load spaCy model
nlp = spacy.load("en_core_web_sm")
def extract_sentence_features_spacy(doc):
    sentences = list(doc.sents)
    tokens = [token for token in doc if not token.is_space]
    tokens_lower = [t.text.lower() for t in tokens]
    uniq_tokens = set(tokens_lower)
    freq_tokens = Counter(tokens_lower)

    if len(sentences) == 0 or len(tokens) == 0:
        return None

    # Sentence length metrics
    sentence_lengths = [len(list(sent)) for sent in sentences]

    # POS tag distribution
    pos_counts = Counter([token.pos_ for token in doc])
    total_tokens = len(tokens)

    # Dependency depth (syntactic complexity)
    def get_tree_depth(token):
        depth = 0
        while token.head != token:
            depth += 1
            token = token.head
        return depth

    dependency_depths = [get_tree_depth(token) for token in doc if not token.is_punct]

    # Entity density
    entities = list(doc.ents)

    features = {
        # Sentence metrics
        'num_sentences': len(sentences),
        'avg_sentence_length': np.mean(sentence_lengths),
        'std_sentence_length': np.std(sentence_lengths),
        'max_sentence_length': np.max(sentence_lengths),
        'min_sentence_length': np.min(sentence_lengths),

        # Token metrics
        'num_tokens': len(tokens),
        'avg_word_length': np.mean([len(token.text) for token in tokens]),
        'std_word_length': np.std([len(token.text) for token in tokens]),

        # Lexical diversity
        'type_token_ratio': len(uniq_tokens) / len(tokens),
        'lemma_diversity': len(set([t.lemma_ for t in tokens])) / len(tokens),
        'hapax_legomenon_density': len([t for t in uniq_tokens if freq_tokens[t] == 1]) / len(uniq_tokens),

        # POS distribution
        'noun_ratio': pos_counts['NOUN'] / total_tokens,
        'verb_ratio': pos_counts['VERB'] / total_tokens,
        'adj_ratio': pos_counts['ADJ'] / total_tokens,
        'adv_ratio': pos_counts['ADV'] / total_tokens,
        'pron_ratio': pos_counts['PRON'] / total_tokens,

        # Syntactic complexity
        'avg_dependency_depth': np.mean(dependency_depths) if dependency_depths else 0,
        'max_dependency_depth': np.max(dependency_depths) if dependency_depths else 0,

        # Entity metrics
        'num_entities': len(entities),
        'entity_density': len(entities) / len(sentences) if len(sentences) > 0 else 0,

        # Readability (using original text)
        'flesch_reading_ease': textstat.flesch_reading_ease(doc.text),
        'flesch_kincaid_grade': textstat.flesch_kincaid_grade(doc.text),

        # Punctuation
        'punctuation_density': sum(1 for t in doc if t.is_punct) / len(tokens),
        'comma_density': sum(1 for t in doc if t.text == ',') / len(tokens),
    }

    return features



In [ ]:
corpus.print_summary_stats()

Number of Speakers: 42202
Number of Utterances: 295027
Number of Conversations: 14371


In [ ]:
c2 = Corpus(download("subreddit-Cornell"))

In [ ]:
c2.print_summary_stats()

Number of Speakers: 7568
Number of Utterances: 74467
Number of Conversations: 10744


In [ ]:
(74467 - 10744)/10744

5.931031273268801

### Rewriting nlp pipeline for efficiency

### AI Response Features

In [9]:
ai_df = pd.read_csv("Dolphin_AmItheAsshole_raw_text.csv")

In [10]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

doc_generator = nlp.pipe(ai_df['ai_response'], n_process=-1, batch_size=50)

In [11]:
sentence_features = []

for doc in doc_generator:
    features = extract_sentence_features_spacy(doc)
    sentence_features.append(features)

In [12]:
ai_features = pd.DataFrame(sentence_features)
ai_features['utterance_id'] = ai_df['id']
ai_features.set_index(['utterance_id'], inplace=True)

In [13]:
ai_features.to_csv('Dolphin_AmItheAsshole_features.csv')

### Human response features

In [ ]:
# Extract human resp features for corpus
corpus = Corpus(download("subreddit-Cornell"))
utt_df = corpus.get_utterances_dataframe()
utt_df = utt_df[utt_df['text'] != '']
#utt_df = utt_df.iloc[:100000]

Dataset already exists at /root/.convokit/saved-corpora/subreddit-Cornell


In [ ]:
sentence_features = []

doc_generator = nlp.pipe(utt_df['text'], n_process=-1, batch_size=50)

for doc in doc_generator:
    features = extract_sentence_features_spacy(doc)
    sentence_features.append(features)



In [ ]:
human_features = pd.DataFrame(sentence_features)


In [ ]:
[i for i in range(len(sentence_features)) if sentence_features[i] is None]

[7870, 70598]

In [ ]:
sentence_features[7870] = {}
sentence_features[70598] = {}

In [ ]:
human_features['utterance_id'] = utt_df.index
human_features.set_index("utterance_id", inplace=True)

In [ ]:
len(human_features) == len(utt_df)

True

In [ ]:
human_features = human_features.drop(human_features.iloc[7870].name)
human_features = human_features.drop(human_features.iloc[70598].name)

In [ ]:
human_features.to_csv("./human_Cornell_features.csv")